<a href="https://colab.research.google.com/github/gmoraissc/Projetos_de_Data_Science/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install xlsxwriter

     |████████████████████████████████| 149 kB 7.2 MB/s 


In [159]:
# imports
import pandas as pd
import numpy as np
import tweepy, nltk, json, os, csv, xlsxwriter
from pandas import ExcelWriter as ExcelWriter
from pathlib import Path
from datetime import datetime, date
from csv import writer

In [3]:
access_token="590375619-tkJialGWR4R5ZoXJfWsZlx0DoLUfSDlbUi1hLATZ"
access_token_secret="cGjIJU3YM23f1qxH8LvUKd6Tm05zbOBUI9NcCDxLzJ39T"
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAHeVYAEAAAAA9sjLNUPQJgclF6kKG0lPvLTq9nY%3DwprwxmnxRvmXpbU2CbXAoIGogKKEZCRxKwoi6xNW3au27KlCqk'
consumer_key= 'vS7GEC5Cy7HixHoVKNpCR7cFr'
consumer_secret="0cpjFY2tWCwDoWhqOwr2ZBDy6TbpNOtWnQOpU70V9V0Bwu0JNt"

In [4]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [90]:
class pesquisas_de_tweets(object):

  def __init__(self):
    pass

  def pesquisar_por_termo(self, query, result_type='recent', count=int(1000)):
    
    global api

    self.query = query
    self.result_type = result_type
    self.count = count

    resultados_da_pesquisa = []

    for status in tweepy.Cursor(api.search,
                                q=self.query,
                                result_type=self.result_type,
                                count=self.count).items():
                                
                                resultados_da_pesquisa.append(status)

    return resultados_da_pesquisa, result_type
 
  def pesquisar_por_usuario(self, part, result_type='recent', count=int(1000)):
    
    global api

    self.part = part
    self.result_type = result_type
    self.count = count
    
    resultados_da_pesquisa = []

    for status in tweepy.Cursor(api.user_timeline,
                                screen_name=self.part,
                                result_type=self.result_type,
                                count=self.count,
                                exclude_replies=False).items():
                                
                                resultados_da_pesquisa.append(status)

    return resultados_da_pesquisa, result_type

In [ ]:
for page in tweepy.Cursor(api.user_timeline).pages():
    # page is a list of statuses
    process_page(page)

In [59]:
def criar_dicionario_pesquisas(tweets_status, tipo):
  
  popular_dict = {}
  recents_dict = {}

  global d1

  for each_json_tweet in tweets_status:
    _ = json.dumps(each_json_tweet._json)

    if tipo == 'popular':
      
      if d1 not in popular_dict.keys():
        tweet = {d1: [json.loads(_)]}
        popular_dict.update(tweet)
    
      else:
        popular_dict[d1].append(json.loads(_))
    
      return popular_dict, 'popular_dict'

    else:
      
      if d1 not in recents_dict.keys():
        tweet = {d1: [json.loads(_)]}
        recents_dict.update(tweet)

      else:
        recents_dict[d1].append(json.loads(_))
        
      return recents_dict, 'recents_dict'

In [60]:
d1 = date.today().strftime("%d/%m/%Y")

In [127]:
def criar_dataframe(dicionario):
  
  lista = []

  for i in range(0, len(dicionario[d1])):

        tweet_id = dicionario[d1][i]['id']
        text = dicionario[d1][i]['text']
        favorite_count = dicionario[d1][i]['favorite_count']
        retweet_count = dicionario[d1][i]['retweet_count']
        created_at = dicionario[d1][i]['created_at']
        location = dicionario[d1][i]['user']['location']
        followers_count = dicionario[d1][i]['user']['followers_count']
        friends_count = dicionario[d1][i]['user']['friends_count']
        verified = dicionario[d1][i]['user']['verified']
        hashtags = dicionario[d1][i]['entities']['hashtags']
        user_mentions = dicionario[d1][i]['entities']['user_mentions']
        name = dicionario[d1][i]['user']['name']
        screen_name = dicionario[d1][i]['user']['screen_name']

        lista.append({'tweet_id': str(tweet_id),
                      'text': str(text),
                      'favorite_count': int(favorite_count),
                      'retweet_count': int(retweet_count),
                      'created_at': created_at,
                      'location': location,
                      'followers_count': followers_count,
                      'friends_count': friends_count,
                      'verified': verified,
                      'retweet_count': retweet_count,
                      'favorite_count': favorite_count,
                      'hashtags': hashtags,
                      'user_mentions': user_mentions,
                      'name': name,
                      'screen_name': screen_name})
        
        tweet_json_ = pd.DataFrame(lista, columns = 
                                  ['tweet_id', 'text', 
                                    'favorite_count', 'retweet_count', 
                                    'created_at', 'location',
                                    'followers_count', 'friends_count',
                                    'verified', 'hashtags',
                                    'user_mentions', 'name',
                                    'screen_name'])
        
        return tweet_json_

In [99]:
tweets, tipo = pesquisas_de_tweets().pesquisar_por_usuario(part='@Naiarazevedo',
                                                         result_type='popular')
a, b = criar_dicionario_pesquisas(tweets, tipo)
criar_dataframe(a)

,tweet_id,text,favorite_count,retweet_count,created_at,location,followers_count,friends_count,verified,hashtags,user_mentions,name,screen_name
0,1483547352644345858,"@contextnaiara Pode sim amor, todos são lindos...",14,0,Tue Jan 18 21:10:26 +0000 2022,"Goiânia, Brasil",149737,1819,True,[],"[{'screen_name': 'contextnaiara', 'name': 'dud...",Naiara Azevedo 💸,Naiarazevedo


In [182]:
def salvar_csvs_no_diretorio(tipo_bbb, tipo_pesquisa, dataframe):
  
  dir_csvs = f'/content/drive/MyDrive/01.Instagram profissional/NLP/Sentiment Analysis/BBB22/Twitter/DataFrames'
  data_extracao = d1.replace('/', '')
  tipo_bbb = 'Camarote'
  tipo_pesquisa = 'Popular'
  file_variable = dir_csvs + "/" + tipo_bbb  + "/" + tipo_pesquisa + "/" + data_extracao
  file_name = tipo_bbb[0:3] + "_" + tipo_pesquisa[0:3] + "_" + data_extracao + '.csv'
  file_path = file_variable + "/" + file_name

  if not os.path.exists(file_variable):
    os.makedirs(file_variable)
    dataframe.to_csv(file_path, index=False)

  else:
    with open(file_path, 'a') as f_object:
      # Pass this file object to csv.writer()
      # and get a writer object
      writer_object = writer(f_object, delimiter=',')

      linhas = []
      
      for row in range(0, len(dataframe)):
        linhas.append([row for row in dataframe.iloc[row]])
        # Pass the list as an argument into
        # the writerow()
        writer_object.writerow(linhas[row])
    
        #Close the file object
      f_object.close()

In [183]:
tweets, tipo = pesquisas_de_tweets().pesquisar_por_termo(query=('Naiara OR BBB22'),
                                                         result_type='popular')
a, b = criar_dicionario_pesquisas(tweets, tipo)
c = criar_dataframe(a)

In [188]:
salvar_csvs_no_diretorio(tipo_bbb='Camarote', tipo_pesquisa='Popular', dataframe=c)

In [189]:
pip install emoji

     |████████████████████████████████| 174 kB 6.6 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=48f47a5c91f4714883f4cf9d5abe266c95e419eb8883edfd2bd813dba620c918
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


In [191]:
termos_de_pesquisa_BBB = ('BBB OR BBB22 OR BIG BROTHER BRASIL OR BIG BROTHER BRASIL 2022 OR OR #REDEBBB OR BIGBROTHER OR #BIGBROTHER OR #BBB OR #BBB22')

termos_de_pesquisa_participantes = {'Arthur': ['Arthur', 'Artur', 'Aguiar', 'Arthur Aguiar'],
                                    'Naiara Azevedo': ['Naiara', 'Azevedo'],
                                    'Pedro Scooby': ['Pedro', 'Scooby'],
                                    'Bruna Gonçalves': ['Bruna', 'Gonçalves', ''],
                                    'Paulo André': ['Paulo', 'André', ''],
                                    'Maria': ['Maria'],
                                    'Jade Picon': ['Jade', 'Picon', '1f32a\ufe0f', 'furacão', '#TeamJade', 'Picão', 'KdJade', 'Picões', 'Furacao'],
                                    'Douglas Silva': ['Douglas', 'Silva', '1f3b2', 'dado', 'dadinho',
                                    '#TeamDouglasSilva', '1f44a\U0001f3ff'],
                                    'Linn da Quebrada': ['Linn', 'Quebrada', '1f9dc\u200d\u2640\ufe0f', '1f9dc\U0001f3ff\u200d\u2640\ufe0f',
                                    '1f9dc\U0001f3fb\u200d\u2640\ufe0f', '1f9dc\U0001f3fe\u200d\u2640\ufe0f', '1f9dc\U0001f3fc\u200d\u2640\ufe0f',
                                    '1f9dc\U0001f3fd\u200d\u2640\ufe0f', '1f9dc', '1f9dc\U0001f3ff', '1f9dc\U0001f3fb', '1f9dc\U0001f3fe',
                                    '1f9dc\U0001f3fc', '1f9dc\U0001f3fd', '#TeamLinn', '#linndonas', '#LinnDonas'],
                                    'Tiago Abravanel': ['Tiago', 'Abravanel', '1f43b', '#TeamAbrava', '#TeamAbravanel', '1f9f8'],
                                    'Laís Caldas': ['Laís', 'Caldas', '1f462', '#TeamLais', '#TeamLaís', 'Time Lais', '1f49a'],
                                    'Luciano Estevan': ['Luciano', 'Estevan', '1f981', '#TeamLucianoEstevan', 'Lu', '1f346'],
                                    'Jessilane': ['Jessilane', '1f9ec', 'Jessi', '#TeamJessi', '#TimeJessi', '1f49c', 'Charmanders', ''],
                                    'Eliezer': ['Eliezer'],
                                    'Eslovênia Marques': ['Eslovênia', 'Eslováquia', 'Marques', 'Eslô', '1f1f8\U0001f1ee',
                                                          'Time Eslo', 'Team Eslo', '#TimeEslô', '#TeamEslô',
                                                          'Eslovenia'],
                                    'Bábara Heck': ['Bárbara', 'Heck', '1f980', '#TeamBá', 'TeamBá', 'TimeBá',
                                                    'TimeBa', 'TeamBa', 'TeamBah', 'BBBah', 'Bá', 'Ba', '1f9a6'],
                                    'Rodrigo Mussi': ['Rodrigo Mussi', 'Mussi', 'Rodrigo', 'TEAM MUSSI', '#TeamMussi',
                                                      '#TimeMussi', '1f977\U0001f3ff', '1f977', '1f977\U0001f3fb',
                                                      '1f977\U0001f3fe', '1f977\U0001f3fc', '1f977\U0001f3fd', 'ninja', 'ninjas',
                                                      'TEAM NINJA', ''],
                                    'Natália Deodato': ['Natália', 'Deodato', '#TeamNaty', '#TimeNaty', 'Naty', 'vitilindos',
                                                        '1f483', '1f483\U0001f3ff', '1f483\U0001f3fb',
                                                        '1f483\U0001f3fe', '1f483\U0001f3fc',
                                                        '1f483\U0001f3fd'],
                                    'Vinicius': ['Vinicius', 'TeamVyni', '#TimeVyni', '#TeamVyni', '1f4a1'],
                                    'Lucas Bissoli': ['Lucas', 'Bissoli', '#TimeBissoli', '#TeamBissoli', 
                                                      '1f3c4\u200d\u2642\ufe0f', '1f3c4\U0001f3ff\u200d\u2642\ufe0f', 
                                                      '1f3c4\U0001f3fb\u200d\u2642\ufe0f', '1f3c4\U0001f3fe\u200d\u2642\ufe0f',
                                                      '1f3c4\U0001f3fc\u200d\u2642\ufe0f', '1f3c4\U0001f3fd\u200d\u2642\ufe0f',
                                                      '1f3c4', '1f3c4\U0001f3ff', '1f3c4\U0001f3fb', '1f3c4\U0001f3fe',
                                                      '1f3c4\U0001f3fc', '1f3c4\U0001f3fd', '1f3c4\u200d\u2640\ufe0f',
                                                      '1f3c4\U0001f3ff\u200d\u2640\ufe0f', '1f3c4\U0001f3fb\u200d\u2640\ufe0f',
                                                      '1f3c4\U0001f3fe\u200d\u2640\ufe0f', '1f3c4\U0001f3fc\u200d\u2640\ufe0f',
                                                      '1f3c4\U0001f3fd\u200d\u2640\ufe0f']}
                                    #teamlinn #emojis #timebissoli

dicionario_participantes = {'@Aguiararthur': 'Arthur Aguiar', 
                            '@Naiarazevedo': 'Naiara Azevedo',
                            '@pedroscooby_': 'Pedro Scooby',
                            '@brunnagoncalves': 'Bruna Gonçalves',
                            '@iampauloandre': 'Paulo André',
                            '@eumaria': 'Maria',
                            '@jadepicon': 'Jade Picon',
                            '@Silva_DG': 'Douglas Silva',
                            '@linndaquebrada': 'Linn da Quebrada',
                            '@TiagoAbravanel': 'Tiago Abravanel',
                            '@Dra_laiscaldass': 'Laís Caldas',
                            '@LucianoEstevan': 'Luciano Estevan',
                            '@a_jessilane': 'Jessilane',
                            '@Eliezerbbb22': 'Eliezer',
                            '@eslomarques': 'Eslovênia Maques',
                            '@bbaheck': 'Bábara Heck',
                            '@oficialmussi': 'Rodrigo Mussi',
                            '@oficial_deodato': 'Natália Deodato',
                            '@vyniof': 'Vinicius',
                            '@LucasBissoli_': 'Lucas Bissoli'}

participantes_camarote = ['@Aguiarthur', '@Naiarazevedo', '@pedroscooby_', '@brunnagoncalves', 
                          '@iampauloandre', '@eumaria', '@jadepicon', '@Silva_DG', '@linndaquebrada', '@TiagoAbravanel']

participantes_pipoca = ['@Dra_laiscaldass', '@LucianoEstevan', '@a_jessilane', '@Eliezerbbb22', '@eslomarques', 
                        '@bbaheck', '@oficialmussi', '@oficial_deodato', '@vyniof', '@LucasBissoli_']

In [ ]:
entrada_e_saida = {'Participante': 'a',
                   'Entrada': data,
                   'Eliminacao': data}

In [140]:
infos_part_pipoca = {}

for part in participantes_pipoca:
  
  infos_part = api.user_timeline(screen_name=part,
                            result_type='recent',
                            count=100)
  
  for each_json_tweet in infos_part:
    
    _ = json.dumps(each_json_tweet._json)
      
    if d1 not in infos_part_pipoca.keys():
      tweet = {d1: [json.loads(_)]}
      infos_part_pipoca.update(tweet)
    
    else:
        infos_part_pipoca[d1].append(json.loads(_))

In [141]:
pipocas_list = []

for i in range(0, len(infos_part_pipoca[d1])):

      tweet_id = infos_part_pipoca[d1][i]['id']
      text = infos_part_pipoca[d1][i]['text']
      favorite_count = infos_part_pipoca[d1][i]['favorite_count']
      retweet_count = infos_part_pipoca[d1][i]['retweet_count']
      created_at = infos_part_pipoca[d1][i]['created_at']
      location = infos_part_pipoca[d1][i]['user']['location']
      followers_count = infos_part_pipoca[d1][i]['user']['followers_count']
      friends_count = infos_part_pipoca[d1][i]['user']['friends_count']
      verified = infos_part_pipoca[d1][i]['user']['verified']
      hashtags = infos_part_pipoca[d1][i]['entities']['hashtags']
      user_mentions = infos_part_pipoca[d1][i]['entities']['user_mentions']
      name = infos_part_pipoca[d1][i]['user']['name']
      screen_name = infos_part_pipoca[d1][i]['user']['screen_name']

      pipocas_list.append({'tweet_id': str(tweet_id),
                            'text': str(text),
                            'favorite_count': int(favorite_count),
                            'retweet_count': int(retweet_count),
                            'created_at': created_at,
                            'location': location,
                            'followers_count': followers_count,
                            'friends_count': friends_count,
                            'verified': verified,
                            'retweet_count': retweet_count,
                            'favorite_count': favorite_count,
                            'hashtags': hashtags,
                            'user_mentions': user_mentions,
                            'name': name,
                            'screen_name': screen_name,
                            })
      
      pipocas_part = pd.DataFrame(pipocas_list, columns = 
                                ['tweet_id', 'text', 
                                  'favorite_count', 'retweet_count', 
                                  'created_at', 'location',
                                  'followers_count', 'friends_count',
                                  'verified', 'hashtags',
                                  'user_mentions', 'name',
                                  'screen_name'])

#pipocas_part.to_csv('/content/drive/MyDrive/01.Instagram profissional/NLP/Sentiment Analysis/BBB22/Twitter/DataFrames/pipocas_17012022')

In [ ]:
infos_part_camarote = {}

for part in participantes_camarote:

  infos_part = api.user_timeline(screen_name=part,
                              result_type='recent',
                              count=100)

  for each_json_tweet in infos_part:
    
    _ = json.dumps(each_json_tweet._json)
      
    if d1 not in infos_part_camarote.keys():
      tweet = {d1: [json.loads(_)]}
      infos_part_camarote.update(tweet)
    
    else:
        infos_part_camarote[d1].append(json.loads(_))

In [ ]:
camarotes_list = []

for i in range(0, len(infos_part_camarote[d1])):

      tweet_id = infos_part_camarote[d1][i]['id']
      text = infos_part_camarote[d1][i]['text']
      favorite_count = infos_part_camarote[d1][i]['favorite_count']
      retweet_count = infos_part_camarote[d1][i]['retweet_count']
      created_at = infos_part_camarote[d1][i]['created_at']
      location = infos_part_camarote[d1][i]['user']['location']
      followers_count = infos_part_camarote[d1][i]['user']['followers_count']
      friends_count = infos_part_camarote[d1][i]['user']['friends_count']
      verified = infos_part_camarote[d1][i]['user']['verified']
      hashtags = infos_part_camarote[d1][i]['entities']['hashtags']
      user_mentions = infos_part_camarote[d1][i]['entities']['user_mentions']
      name = infos_part_camarote[d1][i]['user']['name']
      screen_name = infos_part_camarote[d1][i]['user']['screen_name']

      camarotes_list.append({'tweet_id': str(tweet_id),
                            'text': str(text),
                            'favorite_count': int(favorite_count),
                            'retweet_count': int(retweet_count),
                            'created_at': created_at,
                            'location': location,
                            'followers_count': followers_count,
                            'friends_count': friends_count,
                            'verified': verified,
                            'retweet_count': retweet_count,
                            'favorite_count': favorite_count,
                            'hashtags': hashtags,
                            'user_mentions': user_mentions,
                            'name': name,
                            'screen_name': screen_name,
                            })
      
      camarotes_part = pd.DataFrame(camarotes_list, columns = 
                                ['tweet_id', 'text', 
                                  'favorite_count', 'retweet_count', 
                                  'created_at', 'location',
                                  'followers_count', 'friends_count',
                                  'verified', 'hashtags',
                                  'user_mentions', 'name',
                                  'screen_name'])

camarotes_part.to_csv('/content/drive/MyDrive/01.Instagram profissional/NLP/Sentiment Analysis/BBB22/Twitter/DataFrames/camarote_17012022')

In [ ]:
pipocas_part['screen_name'].unique(), camarotes_part['screen_name'].unique()

(array(['Dra_laiscaldass', 'LucianoEstevan', 'a_jessilane', 'Eliezerbbb22',
        'eslomarques', 'bbaheck', 'oficialmussi', 'oficial_deodato',
        'vyniof', 'LucasBissoli_'], dtype=object),
 array(['Aguiarthur', 'Naiarazevedo', 'pedroscooby_', 'brunnagoncalves',
        'iampauloandre', 'eumaria', 'jadepicon', 'Silva_DG',
        'linndaquebrada', 'TiagoAbravanel'], dtype=object))

In [ ]:
pipocas_part.iloc[pipocas_part['screen_name'] == ''0

,tweet_id,text,favorite_count,retweet_count,created_at,location,followers_count,friends_count,verified,hashtags,user_mentions,name,screen_name
0,1482817681963339777,"🚨 Pela primeira vez, o Globoplay irá transmiti...",28670,831,Sun Jan 16 20:50:59 +0000 2022,Brasil,701265,740,True,"[{'text': 'BBB22', 'indices': [110, 116]}]",[],Tracklist #BBB22,tracklist


In [ ]:
weet_json_recents.tweet_id.dtypes, tweet_json_popular.tweet_id.dtypes

(dtype('O'), dtype('O'))

In [ ]:
tweet_json_recents.join(tweet_json_popular, on='tweet_id')

ValueError: ignored

In [ ]:
calendar = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 
                'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
  
tweet_json['Week_day'] = tweet_json['created_at'].str.slice(0,3)
tweet_json['Month_name'] = tweet_json['created_at'].str.slice(4,7)
tweet_json['Month'] = tweet_json['Month_name'].map(calendar)
tweet_json['Day_number'] = tweet_json['created_at'].str.slice(8,10)
tweet_json['Tweeted_at_time'] = tweet_json['created_at'].str.slice(11,19)
tweet_json['Year'] = tweet_json['created_at'].str.slice(26,30)

NameError: ignored

In [ ]:
tweet_json['Data'] = str(str(tweet_json['Day_number']) + '-' + str(tweet_json['Month']) + '-' + str(tweet_json['Year']))

In [ ]:
tweet_json['Data'][0]

'0     15\n1     14\n2     14\n3     15\n4     14\n5     14\n6     15\n7     15\n8     14\n9     14\n10    15\n11    14\n12    14\n13    14\nName: Day_number, dtype: object-0     1\n1     1\n2     1\n3     1\n4     1\n5     1\n6     1\n7     1\n8     1\n9     1\n10    1\n11    1\n12    1\n13    1\nName: Month, dtype: int64-0     2022\n1     2022\n2     2022\n3     2022\n4     2022\n5     2022\n6     2022\n7     2022\n8     2022\n9     2022\n10    2022\n11    2022\n12    2022\n13    2022\nName: Year, dtype: object'

In [ ]:
# em média, quantos seguidores ganha após um tweet positivo? e quantos perde após um tweet negativo?
# modelo de classificação baseado na popularidade do participante
# prob de ser eliminado no paredão de acordo com o atual nível de popularidade

In [ ]:
pipocas_part.head(3)

,tweet_id,text,favorite_count,retweet_count,created_at,location,followers_count,friends_count,verified,hashtags,user_mentions,name,screen_name
0,1483265947175632897,@medracital ❤️,2,0,Tue Jan 18 02:32:14 +0000 2022,,3247,167,True,[],"[{'screen_name': 'medracital', 'name': 'lore 🇸...",Dra. Laís Caldas 👢,Dra_laiscaldass
1,1483265820528680961,@itscruella Pode encerrar viu? 😅🤍 https://t.co...,2,1,Tue Jan 18 02:31:44 +0000 2022,,3247,167,True,[],"[{'screen_name': 'itscruella', 'name': 'Cruell...",Dra. Laís Caldas 👢,Dra_laiscaldass
2,1483265480945287169,Contem pra gente.. o que estão achando da Laís...,125,9,Tue Jan 18 02:30:23 +0000 2022,,3247,167,True,"[{'text': 'TeamLais', 'indices': [74, 83]}, {'...",[],Dra. Laís Caldas 👢,Dra_laiscaldass


In [ ]:
word_tokenize(pipocas_part['text'])

NameError: ignored